In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
image_dir = 'PATH_TO_RAW_IMAGE_DIR'
label_dir = 'PATH_TO_RAW_LABEL_DIR'
output_image_dir = 'PATH_TO_PROCESSED_IMAGE_DIR'
output_label_dir = 'PATH_TO_PROCESSED_LABEL_DIR'

In [ ]:
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

In [ ]:
IMG_SIZE = 640

In [ ]:
def load_and_resize_image(image_path):
    """Load an image and resize it to the target size."""
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize image to IMG_SIZE
    return img_resized

In [ ]:
def load_and_parse_labels(label_path):
    class_mapping = {
        "Car": 1,
        "Van": 2,
        "Truck": 3,
        "Pedestrian": 4,
        "Person": 5,
        "Cyclist": 6,
        "Motorcyclist": 7,
        "Road_sign": 8
    }
    boxes = []
    with open(label_path, 'r') as file:
        for line in file.readlines():
            parts = line.strip().split()
            if len(parts) < 8:
                continue
            cls_name = parts[0]
            if cls_name in class_mapping:
                cls = class_mapping[cls_name]
            else:
                continue
            try:
                xmin, ymin, xmax, ymax = map(float, parts[4:8])
            except ValueError:
                continue
            boxes.append([cls, xmin, ymin, xmax, ymax])
    return boxes

In [ ]:
image_files = sorted(os.listdir(image_dir))
for image_file in tqdm(image_files, desc="Processing Images"):
    image_path = os.path.join(image_dir, image_file)
    img = cv2.imread(image_path)
    orig_h, orig_w = img.shape[:2]
    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    output_img_path = os.path.join(output_image_dir, image_file)
    cv2.imwrite(output_img_path, img_resized)
    label_file = image_file.replace('.png', '.txt')
    label_path = os.path.join(label_dir, label_file)
    boxes = load_and_parse_labels(label_path)
    x_scale = IMG_SIZE / orig_w
    y_scale = IMG_SIZE / orig_h
    scaled_boxes = []
    for box in boxes:
        cls, xmin, ymin, xmax, ymax = box
        xmin = max(0, min(IMG_SIZE, int(xmin * x_scale)))
        ymin = max(0, min(IMG_SIZE, int(ymin * y_scale)))
        xmax = max(0, min(IMG_SIZE, int(xmax * x_scale)))
        ymax = max(0, min(IMG_SIZE, int(ymax * y_scale)))
        if xmin < xmax and ymin < ymax:
            scaled_boxes.append([cls, xmin, ymin, xmax, ymax])
    output_label_path = os.path.join(output_label_dir, label_file)
    with open(output_label_path, 'w') as out_file:
        for box in scaled_boxes:
            out_file.write(' '.join(map(str, box)) + '\n')
print("Preprocessing complete!")

In [ ]:
import random
def plot_image_with_boxes(image_path, label_path):
    image = cv2.imread(image_path)
    with open(label_path, 'r') as file:
        boxes = [line.strip().split() for line in file.readlines()]
    for box in boxes:
        if len(box) != 5:
            continue
        cls = int(box[0])
        xmin, ymin, xmax, ymax = map(int, map(float, box[1:]))
        if xmin >= xmax or ymin >= ymax:
            continue
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()
preprocessed_image_folder = 'PATH_TO_PROCESSED_IMAGE_DIR'
preprocessed_label_folder = 'PATH_TO_PROCESSED_LABEL_DIR'
image_file = 'PATH_TO_IMAGE_FILE'
image_path = os.path.join(preprocessed_image_folder, image_file)
label_path = os.path.join(preprocessed_label_folder, image_file.replace('.png', '.txt'))
plot_image_with_boxes(image_path, label_path)